In [ ]:
!pip show fastai --version 

In [ ]:
from fastai.vision.all import * 

In [ ]:
path = untar_data(URLs.CAMVID_TINY)

In [ ]:
dls = SegmentationDataLoaders.from_label_func(
    path, 
    bs=8, 
    fnames = get_image_files(path/"images"), 
    label_func = lambda o: path/'labels'/f'{o.stem}_P{o.suffix}',
    codes = np.loadtxt(path/'codes.txt', dtype=str)
)

In [ ]:
# First, load the codes to identify the Void/background class
codes = np.loadtxt(path/'codes.txt', dtype=str)
name2id = {v: k for k, v in enumerate(codes)}
void_code = name2id.get('Void', -1) 

# Define foreground accuracy (ignores background pixels)
def foreground_acc(inp, targ, bkg_idx=void_code, axis=1):
    "Compute accuracy excluding background class"
    targ = targ.squeeze(axis)
    mask = targ != bkg_idx
    return (inp.argmax(dim=axis)[mask] == targ[mask]).float().mean()

In [ ]:
learn = unet_learner(dls, resnet34, metrics=foreground_acc)
# learn = unet_learner(dls, resnet34)

In [ ]:
learn.fine_tune(8)

In [ ]:
learn.show_results(max_n=6)

In [ ]:
doc(dls.bs)